In [ ]:
# Since it worked perfectly with the driver license one, I, again, did ask Chat GPT for the right tags 
# after checking the html structure and making sure I can get all elements with the request.

import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get("https://www.tnwb.uscourts.gov/Search/search.aspx?zoom_sort=0&zoom_xml=0&zoom_query=%22car%22&zoom_per_page=132&zoom_and=1&zoom_cat%5B%5D=-1")
doc = BeautifulSoup(response.text)

In [ ]:
items = doc.select('.result_block, .result_altblock')

In [ ]:
len(items)

In [ ]:
rows = []

for item in items:
    row = {}

    # Case Name
    try:
        row['case_name'] = item.select_one('.result_title a').text.strip()
    except:
        row['case_name'] = 'no name'

    # URL to the Case (PDF)
    try:
        row['url'] = item.select_one('.result_title a')['href']
    except:
        row['url'] = None

    # Category
    try:
        row['category'] = item.select_one('.result_title .category').text.strip(" []")
    except:
        row['category'] = None

    # Additional Details (Terms matched, size, PDF URL)
    
    try:
        detail_text = item.select_one('.infoline').text.strip()
        parts = detail_text.split(' - ')
    
    # Terms matched
        row['terms_matched'] = int(parts[0].replace('Terms matched:', '').strip())
    
    # File size
        row['file_size'] = parts[1].strip()
    
    # PDF URL
        row['pdf_url'] = parts[2].replace('URL:', '').strip()
    except:
        row['terms_matched'] = None
        row['file_size'] = None
        row['pdf_url'] = None

    rows.append(row)


In [ ]:
len(rows)

In [ ]:
rows[0]

In [ ]:
df = pd.json_normalize(rows)

In [ ]:
df.to_csv('court_cases.csv')